<a href="https://colab.research.google.com/github/cerr/CERR/blob/octave_dev/Jupyter_Notebooks/demo_OMT_distance_between_dose_distributions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Install Octave 
---



In [ ]:
%%capture 
!! apt-get update
! apt-get install gcc g++ gfortran make libblas-dev liblapack-dev libpcre3-dev libarpack2-dev libcurl4-gnutls-dev epstool libfftw3-dev transfig libfltk1.3-dev libfontconfig1-dev libfreetype6-dev libgl2ps-dev libglpk-dev libreadline-dev gnuplot-x11 libgraphicsmagick++1-dev libhdf5-serial-dev openjdk-8-jdk libsndfile1-dev llvm-dev lpr texinfo libgl1-mesa-dev libosmesa6-dev pstoedit portaudio19-dev libqhull-dev libqrupdate-dev libqscintilla2-dev libsuitesparse-dev texlive texlive-generic-recommended libxft-dev zlib1g-dev autoconf automake bison flex gperf gzip icoutils librsvg2-bin libtool perl rsync tar qtbase5-dev qttools5-dev qttools5-dev-tools libqscintilla2-qt5-dev wget git libsundials-dev gnuplot x11-apps


In [ ]:
%%capture
%%bash
LOCATION=$(curl -s https://api.github.com/repos/cerr/octave-colab/tags \
| grep "name" | tail -n 1 \
| awk '{print "https://github.com/cerr/octave-colab/archive/" substr($2, 2, length($2)-3) ".zip"}') \
; curl -L -o octavecolab.zip $LOCATION

unzip octavecolab.zip -d octavecolab
releaseDir=$(ls octavecolab)
tar xzvf "octavecolab/${releaseDir}/octavecolab.tar.gz"

In [ ]:
import os
os.environ['OCTAVE_EXECUTABLE'] = '/content/octavecolab/bin/octave-cli'
os.environ['PATH'] = '/content/octavecolab/bin:' + os.environ['PATH']

## Install Oct2Py and required packages

In [ ]:
%%capture
!pip3 install octave_kernel
!pip3 install oct2py

%load_ext oct2py.ipython

!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/image-2.12.0.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/io-2.6.1.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/statistics-1.4.2.tar.gz

!octave --eval "pkg install image-2.12.0.tar.gz"
!octave --eval "pkg install io-2.6.1.tar.gz"
!octave --eval "pkg install statistics-1.4.2.tar.gz"

## Download CERRx

In [ ]:
%%capture
!git clone --single-branch --branch octave_dev https://www.github.com/cerr/CERR.git

## Sample OMT distance calculations

In [ ]:
%%capture
from oct2py import octave

octave.eval('pkg load image')
octave.eval('pkg load io')
octave.eval('pkg load statistics')
octave.addpath(octave.genpath('/content/CERR'))

In [ ]:
#Load sample data
sampleData = '/content/CERR/Unit_Testing/data_for_cerr_tests/CERR_plans/' + \
             'lung_ex1_20may03.mat.bz2'

%octave_push sampleData

planC = octave.loadPlanC(sampleData,octave.tempdir());
planC = octave.updatePlanFields(planC);
planC = octave.quality_assure_planC(sampleData,planC);

In [ ]:
# Example 1. Sanity check (distance between identical distributions)

doseNum1 = 1;
doseNum2 = 1;
structNum = 3;
gamma = 0.1;
downsampleIndex = 1;

dist1 = octave.calcOMTDoseDistance(doseNum1,doseNum2,structNum,gamma,downsampleIndex,planC);
fprintf('OMT distance = ',dist1)

In [ ]:
# Example 2. Distance between two dose distributions in region of interest

doseNum1 = 1;
doseNum2 = 2;
structNum = 3; #PTV
gamma = 0.1;
downsampleIndex = 1;

dist1 = octave.calcOMTDoseDistance(doseNum1,doseNum2,structNum,gamma,downsampleIndex,planC);
print(dist1)

0.012116394947413286


In [ ]:
from oct2py import octave
%octave_push planC

In [ ]:
%%octave

addpath(genpath('/content/CERR'));

indexS = planC{end};
scanType = 'normal';
structNum = 3;
doseNum1 = 1;
doseNum2 = 2;
scanNum = getStructureAssociatedScan(structNum,planC);
scan3M = getScanArray(scanNum,planC);
 
%Get dose arrays on CT grid
dose1M = getDoseOnCT(doseNum1, scanNum, scanType, planC);
dose2M = getDoseOnCT(doseNum2, scanNum, scanType, planC);

% Crop doses
struct3M = getStrMask(structNum,planC);
[rMin,rMax,cMin,cMax,sMin,sMax] = compute_boundingbox(struct3M);
dose1M(~struct3M) = 0;
dose2M(~struct3M) = 0;
dose1M = dose1M(rMin:rMax,cMin:cMax,sMin:sMax);
dose2M = dose2M(rMin:rMax,cMin:cMax,sMin:sMax);

In [ ]:
%octave_pull dose1M dose2M scan3M